In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import time
import h5py
import os,sys
import numpy as np

In [3]:
DSPACK_HOME=os.environ.get("DSPACK_HOME")
if DSPACK_HOME not in sys.path:
    sys.path.append(DSPACK_HOME)

In [4]:
import scripts.fsystem
import scripts.statistics
reload(scripts.statistics)

<module 'scripts.statistics' from '/reg/data/ana03/scratch/zhensu/Develop/psvolume/v3.0/psvolume/scripts/statistics.pyc'>

In [5]:
workdir = "/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225/"
dnames = ["WT-1", "WT-2", "WT-3", "G150A-1", "G150A-2", "G150A-3", "G150T-1", "G150T-2", "G150T-3"]

In [6]:
def get_processing_results(pipeline="standard",scale_factor="profile",pca_step=True,dnames=None,workdir=None):
    data = np.zeros((5,9))
    file_tail = "_with_pca_map1.dsdata" if pca_step else "_map1.dsdata"
    
    ## Get CCFriedel
    for idx,dname in enumerate(dnames):
        dsdata_file = "%s/data/%s/%s_clean_data_scale_%s"%(workdir,dname,pipeline,scale_factor)+file_tail
        map_file = scripts.fsystem.H5manager.reader(dsdata_file,"merge_volume")
        pdb_file = scripts.fsystem.H5manager.reader(dsdata_file,"pdb_refmac5")
        data[0, idx] = scripts.statistics.calculate_ccfriedel_from_map(map_file,pdb_file)
        if dname=="WT-1":
            print dsdata_file
        
    ## Get CCLaue
    for idx,dname in enumerate(dnames):
        dsdata_file = "%s/data/%s/%s_clean_data_scale_%s"%(workdir,dname,pipeline,scale_factor)+file_tail
        map_file = scripts.fsystem.H5manager.reader(dsdata_file,"merge_volume")
        pdb_file = scripts.fsystem.H5manager.reader(dsdata_file,"pdb_refmac5")
        data[1, idx] = scripts.statistics.calculate_cclaue_from_map(map_file,pdb_file)
        if dname=="WT-1":
            print dsdata_file
        
    ## Get CC1/2
    for idx,dname in enumerate(dnames):
        dsdata_file = "%s/data/%s/%s_clean_data_scale_%s"%(workdir,dname,pipeline,scale_factor)+file_tail
        phenix_file = scripts.fsystem.H5manager.reader(dsdata_file,"phenix_merge_stats")
        data[2, idx] = scripts.fsystem.H5manager.reader(phenix_file,"phenix_cc12")
        if dname=="WT-1":
            print dsdata_file
        
    ## Get CCRep
    marker = np.triu(np.ones((9,9)).astype(int), 1)
    marker[0:6,6:9] = 0
    marker[0:3,3:6] = 0
    
    map_file_tail = "_with_pca.out" if pca_step else ".out"
    path_cc_map = "%s/data/CC-MAP/ccmap_%s_clean_data_scale_%s"%(workdir,pipeline,scale_factor)+map_file_tail
    ccmap = scripts.fsystem.H5manager.reader(path_cc_map,"cc")
    ccmap *= marker
    data[3] = np.sum(np.maximum(ccmap,ccmap.T),axis=0) / np.sum(np.maximum(marker,marker.T),axis=0)
    print path_cc_map
    
    ## Get CCLLM (anisotropic ADP)
    for idx, dname in enumerate(dnames):
        path_dsdata = "%s/data/%s/%s_clean_data_scale_%s"%(workdir,dname,pipeline,scale_factor)+file_tail
        path_llm = scripts.fsystem.H5manager.reader(path_dsdata,"lunus_llm_refmac5")
        with h5py.File(path_llm,"r") as f:
            data[4,idx] = f["lunus_llm_aniso_cc"][()]
        if dname=="WT-1":
            print path_dsdata
            
    return data

In [7]:
def print_data(data,ref):
    data_name = ["CC_Friedel","CC_Laue","CC_1/2","CC_Rep","CC_LLM"]
    for i in range(len(data)):
        x = []
        for j in range(len(data.T)):
            value = "{:.2f}".format(data[i,j] - ref[i,j])
            if data[i,j] - ref[i,j] > 0:
                value = "+"+value
            x.append(value)
        print str(data_name[i]).ljust(12), " ".join([_.ljust(8) for _ in x])

In [8]:
data_name = ["CC_Friedel","CC_Laue","CC_1/2","CC_Rep","CC_LLM"]

## A. Standard Pipeline

In [10]:
data = get_processing_results(pipeline="standard",scale_factor="profile",pca_step=True,dnames=dnames,workdir=workdir)
ref = data.copy()
## Print the data
for idx,x in enumerate(data):
    print str(data_name[idx]).ljust(12)," ".join([str(round(i,2)).ljust(8) for i in x])

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_standard_clean_data_scale_profile_with_pca.out
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_with_pca_map1.dsdata
CC_Friedel   0.93     0.91     0.91     0.93     0.91     0.91     0.91     0.92     0.94    
CC_Laue      0.9      0.87     0.87     0.88     0.86     0.85     0.86     0.88     0.91    
CC_1/2       0.85     0.78     0.81     0.81     0.76     0.77     0.82     0.84     0.89    
CC_Rep       0.86     0.84     0.85     0.82     0.81     0.82     0.88     0.87     0.89    
CC_LLM       0.65  

## B. Without Non-Crystal Background Subtraction

In [11]:
data = get_processing_results(pipeline="nobackg",scale_factor="profile",pca_step=True,dnames=dnames,workdir=workdir)
print_data(data,ref)

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nobackg_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nobackg_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nobackg_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_nobackg_clean_data_scale_profile_with_pca.out
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nobackg_clean_data_scale_profile_with_pca_map1.dsdata
CC_Friedel   -0.01    -0.01    -0.02    -0.01    -0.03    -0.02    -0.03    -0.02    -0.02   
CC_Laue      -0.01    -0.03    -0.02    -0.02    -0.03    -0.03    -0.03    -0.04    -0.02   
CC_1/2       -0.02    -0.06    -0.04    -0.04    -0.07    -0.06    -0.06    -0.06    -0.03   
CC_Rep       -0.03    -0.05    -0.05    -0.08    -0.08    -0.05    -0.03    -0.05    -0.03   
CC_LLM       -0.00    -0

## C. Without Polarization Correction

In [12]:
data = get_processing_results(pipeline="nopolar",scale_factor="profile",pca_step=True,dnames=dnames,workdir=workdir)
print_data(data,ref)

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nopolar_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nopolar_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nopolar_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_nopolar_clean_data_scale_profile_with_pca.out
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nopolar_clean_data_scale_profile_with_pca_map1.dsdata
CC_Friedel   +0.04    +0.04    +0.04    +0.03    +0.05    +0.04    +0.03    +0.03    +0.02   
CC_Laue      +0.05    -0.02    +0.04    -0.01    +0.07    -0.03    +0.03    -0.11    +0.03   
CC_1/2       +0.08    -0.06    +0.08    -0.06    +0.13    -0.11    +0.02    -0.19    +0.04   
CC_Rep       -0.13    -0.32    -0.27    -0.58    -0.31    -0.32    -0.16    -0.35    -0.15   
CC_LLM       -0.12    -0

## D. Without Radial Profile Variance Removal

In [13]:
data = get_processing_results(pipeline="standard",scale_factor="profile",pca_step=False,dnames=dnames,workdir=workdir)
print_data(data,ref)

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_standard_clean_data_scale_profile.out
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/standard_clean_data_scale_profile_map1.dsdata
CC_Friedel   -0.01    -0.01    -0.02    -0.02    -0.01    -0.03    -0.02    -0.03    -0.00   
CC_Laue      -0.02    -0.06    -0.01    -0.05    -0.10    -0.07    -0.03    -0.04    -0.03   
CC_1/2       -0.04    -0.11    -0.03    -0.09    -0.18    -0.15    -0.05    -0.07    -0.04   
CC_Rep       -0.05    -0.08    -0.11    -0.04    -0.02    -0.04    -0.01    -0.02    -0.01   
CC_LLM       +0.01    -0.01    +0.01    -0.01    -0.01    -0.05 

## E. Without Solid-Angle Correction

In [14]:
data = get_processing_results(pipeline="nosolid",scale_factor="profile",pca_step=True,dnames=dnames,workdir=workdir)
print_data(data,ref)

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nosolid_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nosolid_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nosolid_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_nosolid_clean_data_scale_profile_with_pca.out
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/nosolid_clean_data_scale_profile_with_pca_map1.dsdata
CC_Friedel   +0.00    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01   
CC_Laue      +0.01    +0.02    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01   
CC_1/2       +0.01    +0.03    +0.02    +0.02    +0.02    +0.02    +0.03    +0.02    +0.01   
CC_Rep       +0.01    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01    +0.01   
CC_LLM       +0.01    +0

## F. Without Detector Absorption Correction

In [15]:
data = get_processing_results(pipeline="noabsorp",scale_factor="profile",pca_step=True,dnames=dnames,workdir=workdir)
print_data(data,ref)

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noabsorp_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noabsorp_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noabsorp_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_noabsorp_clean_data_scale_profile_with_pca.out
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noabsorp_clean_data_scale_profile_with_pca_map1.dsdata
CC_Friedel   -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00   
CC_Laue      -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00   
CC_1/2       -0.00    -0.01    -0.01    -0.01    -0.01    -0.01    -0.01    -0.00    -0.00   
CC_Rep       -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00    -0.00   
CC_LLM       -0.00 

## G. Without Parallax Correction

In [16]:
data = get_processing_results(pipeline="noparallax",scale_factor="profile",pca_step=True,dnames=dnames,workdir=workdir)
print_data(data,ref)

/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noparallax_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noparallax_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noparallax_clean_data_scale_profile_with_pca_map1.dsdata
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/CC-MAP/ccmap_noparallax_clean_data_scale_profile_with_pca.out
/reg/data/ana03/scratch/zhensu/Experiment/ICH/20201225//data/WT-1/noparallax_clean_data_scale_profile_with_pca_map1.dsdata
CC_Friedel   +0.00    -0.00    +0.00    -0.00    -0.00    -0.00    -0.00    +0.00    -0.00   
CC_Laue      +0.00    -0.00    +0.00    -0.00    -0.00    -0.00    -0.00    +0.00    -0.00   
CC_1/2       +0.00    0.00     0.00     -0.00    0.00     +0.00    -0.00    -0.00    -0.00   
CC_Rep       +0.00    +0.00    +0.00    -0.00    -0.00    -0.00    +0.00    +0.00    -0.00   
CC_LLM   